<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
from datetime import datetime
import re
import sys
sys.path.append('../')

In [2]:
from rgtfs.helpers import treat_rj_brt_realized_trips

/usr/local/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
import rgtfs
from pathlib import Path

In [10]:
raw_path = 'tmp'

In [17]:
treat_rj_brt_realized_trips(raw_path)

,vehicle_id,departure_datetime,arrival_datetime,departure_id,arrival_id,distance,elapsed_time,average_speed,trajectory_type,trip_id,trip_short_name
3,30540,2021-04-01 05:49:36,2021-04-01 06:22:50,None,None,20.60,00:33:14,37.45,complete_trip,None,18 - RECREIO SHOPPING X JARDIM OCEÂNICO ( EXPR...
5,30509,2021-04-01 06:17:22,2021-04-01 06:50:19,None,None,20.60,00:32:57,38.63,complete_trip,None,18 - RECREIO SHOPPING X JARDIM OCEÂNICO ( EXPR...
7,47031,2021-04-01 06:44:29,2021-04-01 07:18:12,None,None,20.60,00:33:43,37.45,complete_trip,None,18 - RECREIO SHOPPING X JARDIM OCEÂNICO ( EXPR...
8,30540,2021-04-01 07:04:23,2021-04-01 07:39:29,None,None,20.60,00:35:06,35.31,complete_trip,None,18 - RECREIO SHOPPING X JARDIM OCEÂNICO ( EXPR...
9,30509,2021-04-01 07:31:54,2021-04-01 08:06:10,None,None,20.60,00:34:16,36.35,complete_trip,None,18 - RECREIO SHOPPING X JARDIM OCEÂNICO ( EXPR...
...,...,...,...,...,...,...,...,...,...,...,...
3455,47027,2021-04-02 00:35:46,2021-04-02 00:40:28,None,None,14.76,00:04:42,NaN,not_complete_trip,None,50 - TERMINAL OLIMPICO X J.OCEÂNICO (PARADOR)
3541,13401,2021-04-02 01:47:24,2021-04-02 01:50:40,None,None,25.40,00:03:16,NaN,not_complete_trip,None,19 - SALVADOR ALLENDE X PINGO D'AGUA ( EXPRESSO )
3645,44738,2021-04-02 10:14:08,2021-04-02 12:17:59,None,None,28.80,02:03:51,14.05,complete_trip,None,46- PENHA X ALVORADA ( EXPRESSO )
3662,44738,2021-04-02 14:08:27,2021-04-02 14:13:08,None,None,28.80,00:04:41,NaN,not_complete_trip,None,46- PENHA X ALVORADA ( EXPRESSO )


In [143]:
df = pd.concat([treat_rj_brt_realized_trips(p) for p in Path('../data/raw/sonda_brt/').glob('*.csv')])  

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning:

Columns (3,15) have mixed types.Specify dtype option on import or set low_memory=False.

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning:

Columns (3,15) have mixed types.Specify dtype option on import or set low_memory=False.

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning:

Columns (3,15) have mixed types.Specify dtype option on import or set low_memory=False.

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning:

Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.



In [144]:
df.to_csv('../data/output/realized_trips_brt.csv', index=False)

In [145]:
import plotly.express as px

In [146]:
brt = pd.read_csv('../data/output/realized_trips_brt.csv', parse_dates=['departure_datetime'])

In [147]:
gtfs = pd.read_csv('../data/output/realized_trips_gtfs.csv', parse_dates=['departure_datetime'])

In [148]:
def hourly_trips(df, label):

    df = df.query('trajectory_type == "complete_trip"')
    df['departure_hourly'] = df['departure_datetime'].apply(lambda x: x.replace(minute=0, second=0))
    df = df.groupby('departure_hourly').count()[['arrival_datetime']].rename(columns={'arrival_datetime': 'n_trips'})
    df['label'] = label
    return df.reset_index()

In [149]:
# r = hourly_trips(rgtfs, 'Executado (GPS)')
# g = hourly_trips(gtfs, 'Planejado (GTFS)')
b = hourly_trips(brt, 'Executado (BRT)')
fig = px.line(pd.concat([b]), x='departure_hourly', y='n_trips', color='label')

fig.update_layout(
    template='simple_white',
    yaxis_title='Número de Viagens',
    title='Número de Viagens BRT por Hora (Executado)',
    hovermode="x"
)

fig.show()

<ipython-input-148-279f7f1e3996>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [159]:
df = pd.merge(b, g, on='departure_hourly', suffixes=['_brt', '_gtfs'])

In [160]:
df['Número de Viagens'] = df['n_trips_gtfs'] - df['n_trips_brt']

In [161]:
df['departure_daily'] = df['departure_hourly'].apply(lambda x: x.replace(hour=0))

In [162]:
d = df.groupby('departure_daily').sum().reset_index()

In [164]:
px.line(d, x='departure_daily', y='n_trips_brt',
        title='Número de Viagens BRT por Dia (Realizado/SONDA)')

In [154]:
d['dow'] = d['departure_daily'].apply(lambda x: x.dayofweek)

In [155]:
d = d[~d['dow'].isin([5,6])]

In [156]:
px.line(d, x='departure_daily', y='Número de Viagens',
        title='Diferença entre Planejado e Executado por Dia (Dias Úteis)')